### Set Covering
To rapresent states is a good procedure to use sets instead of arrays and vectors because sets are unordered.

We need a set for the tiles we need and one for the tiles we don't need.

The problem we are trying to solve is to find a state (which is a set of sets of True and False which means)

In [18]:
from random import random
from functools import reduce
import numpy as np
from queue import PriorityQueue, LifoQueue, SimpleQueue
from threading import Thread
import time

PROBLEM_SIZE=10
NUM_SETS=40
SETS=tuple(np.array([random() < .1 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))

def goal_check(state):
    return np.all(reduce(np.logical_or,[SETS[i] for i in state[0]],np.array([False for _ in range(PROBLEM_SIZE)])))

def get_covered(state):
    return reduce(np.logical_or,[SETS[i] for i in state[0]],np.array([False for _ in range(PROBLEM_SIZE)]))

def get_distance_from_state(state, action): # Cost For Greedy Solution
    covered=get_covered(state)

    res=np.logical_and(SETS[action], np.logical_not(covered))
    # if np.count_nonzero(t==True):
    #     print(f"State: {t}")
    #     print(f"Action: {SETS[action]}")
    #     print(f"Res: {res}")
    return np.count_nonzero(res==True)


def get_distance_from_goal(state):  #Cost (Euristhic) For Informed Solution
    t=get_covered(state)
    return np.count_nonzero(t==False)
    
def get_common_elements(state, action):
    covered=get_covered(state)
    temp=np.count_nonzero(np.logical_and(covered, SETS[action])==True)
    return temp
def get_largest_tile(state):
    covered=get_covered(state)

    d=0
    for _,i in enumerate(state[1]):
        temp=np.count_nonzero( np.logical_and(SETS[i], np.logical_not(covered)) ==True)
        if(temp>d):
            d=temp
    return d


def state_to_str(state):
    c=get_covered(state)
    string=""
    for i in c:
        string+="*" if i else "_"
    return string

def set_to_str(set):
    string=""
    for i in set:
        string+="*" if i else "_"
    return string


In the following code there are 4 possible solutions to achieve the goal. 
- Greedy finds the "best solution" in the frontieer
- Informed findes the "closest solution" to the solution while not considering the cost
- A* simple also consider a unit cost 
- A* complicated considers the best solution in the frontieer that is closer to the solution

In [19]:
def Solve( case):

    start=time.time()
    frontier=PriorityQueue()
    frontier.put((0,[set(), set(range(NUM_SETS))] , 0))

    t=frontier.get()
    current_state=t[1]
    print(state_to_str(current_state))
    
    counter=0
    name=""

    if case==0:
        name="Greedy"
    if case==1:
        name="Informed"
    if case==2:
        name="A* simple"
    if case==3:
        name="A* complicated"
    v=0
    while not goal_check(current_state):
        
        counter+=1
        
        largest_tile=get_largest_tile(current_state)
        
        for action in current_state[1]:
            new_state=(current_state[0] | {action}, current_state[1] - {action})

            # --- Greedy Solution --- #
            if(case==0):
                d_to_next_state=t[2]- get_distance_from_state(current_state,action)
                frontier.put((d_to_next_state,new_state, d_to_next_state))

            # --- Informed Solution --- #
            if(case==1):
                frontier.put((get_distance_from_goal(new_state), new_state, t[2]))

            # --- A* easy Solution --- #
            if(case==2):
                
                if largest_tile!=0:
                    v=get_distance_from_goal(new_state)//largest_tile + len(new_state[0])

                    frontier.put((v, new_state , v))

            # --- A* wrong Solution  --- #
            if(case==3):
                d_to_next_state=t[2]- get_distance_from_state(current_state,action)
                if(d_to_next_state!=0):
                    frontier.put((  d_to_next_state+len(new_state[0]), new_state, d_to_next_state))
            

        if goal_check(current_state):
            break

        t=frontier.get()
        current_state=t[1]
    
    end=time.time()-start
    
    print(f"Solved {name} in {counter} steps with {len(new_state[0])} tiles in {end} ms\n ")
    
state=[set(range(NUM_SETS)), set()]


The cost implemented for greedy solutions values as more expensive a step with lower number of "True" added to the state.

In [20]:
assert goal_check(state),"Not Solvable Problem"

Solve(0)
Solve(1)
Solve(3)
Solve(2)
# t1=Thread(target=Solve, args=[0])
# t2=Thread(target=Solve, args=[1])
# t3=Thread(target=Solve, args=[2])
# t4=Thread(target=Solve, args=[3])

# t1.start()
# t2.start()
# t3.start()
# t4.start()

# t1.join()
# t2.join()
# t3.join()
# t4.join()


__________
Solved Greedy in 6 steps with 6 tiles in 0.004999876022338867 ms
 
__________
Solved Informed in 6 steps with 6 tiles in 0.0030012130737304688 ms
 
__________


: 

To confront the solutions I'm starting different thread that measure the time between the start and the end of the algorithm (with the same set of sets). \\ 
It is possible to notice that in this case A* simple usually solves the problem in 1 or 2 steps more than the others.

## A* correct solution

Actually the Heuristic chosen for A* is not admissible, hencefor what has been said is not valid. \
A possible herusitic is looking for the largest set and dividing the remaining "True"s by that size.\
For this reason at each iteration the heuristic is calculated as the remaining elements to cover over the maximum$\frac{get_distance_from_goal()} $

## A* wrong

Among the solution the A* wrong solution takes into account both the distance from goal (how many falses are there) and the distance from the previous state, which still leads to a solution, but the solution isn't optimal and is not A*, since the heursitic is not admissible and the cost easly overcomes the heuristic, thus it mostly behaves as a greedy solution
